# 模块B：智能负载均衡（Balancer）测试

覆盖：负载均衡算法、健康检查（HTTP/AI/脚本）、后端管理/连接池、服务发现/动态注册、故障转移与预热。

提示：默认使用 `../build` 目录；如需重新编译，把 `BUILD_FIRST = True`。

In [ ]:
from __future__ import annotations

import os
import subprocess
from pathlib import Path

ROOT = (Path("..") if Path.cwd().name == "notebooks" else Path(".")).resolve()
BUILD_DIR = (ROOT / "build").resolve()

TIMEOUT_S = 30
BUILD_FIRST = False
CONTINUE_ON_FAIL = False

TESTS = [
    # core algorithms
    "test_balancer",
    "test_leastconn",
    "test_leastqueue",
    "test_rtw",
    "test_gpu_balancer",
    # health checks
    "test_balancer_health",
    "test_http_health_checker",
    "test_ai_service_check",
    "test_script_health_checker",
    "test_warmup",
    # backend management / connection pool
    "test_backend_manager",
    "test_backend_conn_pool",
    # routing features
    "test_model_affinity",
    # discovery / register
    "test_service_discovery",
    "test_dynamic_register",
    # failover
    "test_passive_failover",
]


In [ ]:
def sh(cmd: list[str], *, cwd: Path = ROOT, timeout: int | None = None) -> None:
    print("$", " ".join(cmd))
    subprocess.run(cmd, cwd=str(cwd), check=True, timeout=timeout)


def maybe_build() -> None:
    BUILD_DIR.mkdir(parents=True, exist_ok=True)
    cache = BUILD_DIR / "CMakeCache.txt"
    if not cache.exists():
        sh(["cmake", "-S", str(ROOT), "-B", str(BUILD_DIR)])
    jobs = str(max(1, os.cpu_count() or 4))
    sh(["cmake", "--build", str(BUILD_DIR), "-j", jobs])


def run_tests() -> None:
    failures: list[tuple[str, str]] = []
    for t in TESTS:
        p = BUILD_DIR / t
        if not p.exists():
            print(f"SKIP {t} (missing: {p})")
            continue
        print(f"RUN {t}")
        try:
            sh([str(p)], cwd=ROOT, timeout=TIMEOUT_S)
        except subprocess.TimeoutExpired:
            print(f"FAIL {t} timeout={TIMEOUT_S}s")
            failures.append((t, "timeout"))
            if not CONTINUE_ON_FAIL:
                break
        except subprocess.CalledProcessError as e:
            print(f"FAIL {t} exit={e.returncode}")
            failures.append((t, f"exit={e.returncode}"))
            if not CONTINUE_ON_FAIL:
                break

    if failures:
        raise RuntimeError("Failed tests: " + ", ".join([f"{n}({r})" for n, r in failures]))
    print("OK")


if BUILD_FIRST:
    maybe_build()
run_tests()
